In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from preprocessing import load_gni_pc, load_events, load_bing_results, check_counts_and_merge, load_gni_class, load_gni_region,load_population, load_gdp, load_if_in_country_or_lang, load_views
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## load df

In [4]:
df = load_events(drop_zeroes=False)
df_news = load_bing_results()
df_crawled = check_counts_and_merge(df, df_news, dropna=False)
# show article counts

Loading de
LOAD FROM scraping/results/de_bingnews.json, Articles now: 2304
LOAD FROM scraping/results/de_bingnews_missing.json, Articles now: 2376
LOAD FROM scraping/results/de_bingnews_0.json, Articles now: 4152
Loading en
LOAD FROM scraping/results/en_bingnews.json, Articles now: 5262
LOAD FROM scraping/results/en_bingnews_rest.json, Articles now: 7761
LOAD FROM scraping/results/en_bingnews_missing.json, Articles now: 7761
LOAD FROM scraping/results/en_bingnews_0.json, Articles now: 9182
Loading it
LOAD FROM scraping/results/it_bingnews.json, Articles now: 2297
LOAD FROM scraping/results/it_bingnews_missing.json, Articles now: 2358
LOAD FROM scraping/results/it_bingnews_0.json, Articles now: 3622
Loading es
LOAD FROM scraping/results/es_bingnews_es-ES.json, Articles now: 3253
LOAD FROM scraping/results/es_bingnews_es-ES_missing.json, Articles now: 3448
LOAD FROM scraping/results/es_bingnews_0.json, Articles now: 4292
Loading es
LOAD FROM scraping/results/es_bingnews_es-ES_corrected.j

In [5]:
df_crawled[pd.isna(df_crawled.bing_hits) & pd.isna(df_crawled.bing_hits)].country.unique()

array(['Taiwan', 'Brunei', 'Ireland', 'Jersey', 'France', 'China',
       'Pitcairn Islands', 'Faeroe Islands', 'Cook Islands', 'Gibraltar'],
      dtype=object)

## load gni region and regional class

In [6]:
df_crawled, df_missing_class = load_gni_class(df_crawled)
df_crawled, df_missing_region = load_gni_region(df_crawled)


Lost Events for class: 28
Lost Events for region:  0


In [7]:
df_missing_class.country.unique()

array(['Falkland Islands', 'Faeroe Islands', 'Brunei', 'Jersey',
       'Cook Islands', 'Pitcairn Islands'], dtype=object)

## load population

In [8]:
df_crawled, df_missing_pop = load_population(df_crawled)

Lost Events for population:  0


# Merge with GDP

In [9]:
df_crawled, df_missing_gdp = load_gdp(df_crawled)

Lost Events for gdp:  4


In [10]:
df_missing_gdp.country.unique()

array(['Gibraltar'], dtype=object)

# compute lang and code assignment

In [11]:
df_crawled = load_if_in_country_or_lang(df_crawled)

# Store resulting dataframe, then change manually which events were not found

Note that these are commented out because we already did that once. the manually processed df is fine from here on out.

In [ ]:
filter_disaster = (df_crawled.cat == 'undefined')
df_crawled[filter_disaster].to_csv('events/processed_0_undefined.csv', index=False)
df_crawled[~filter_disaster].to_csv('events/processed_0.csv', index=False)

In [21]:
# now, fix undefined by hand, and reload
df_fixed = pd.read_csv('events/processed_0_fixed.csv')

In [ ]:
df_new = pd.concat([df_crawled[~filter_disaster], df_fixed])

In [ ]:
### Store missing bing-results
for code, df in df_new.groupby('code'):
    np.savetxt(f'scraping/scripts/{code}_articles_0.txt', df[pd.isna(df.bing_hits)].pagetitle.values, fmt='%s')

# Merge with views

In [12]:
from preprocessing import compute_view_baseline
from preprocessing import replace_country_names

In [14]:
#df_new = pd.read_csv('events/new/processed_manually.csv')
df_new = df_crawled
df_new.event_date = pd.to_datetime(df_new.event_date)
df_new['date_month'] = df_new.event_date.apply(lambda d: pd.to_datetime(f'{d.year}-{d.month:02d}'))
df_new.head()

,event_id,pagetitle,event_date,economic_region,country,continent,cat,categories,views_7_sum,views_before_mean,...,gni_region,population,Country/Area,Year,GDP_pc,GDP,oecd,in_code_lang,in_code_region,date_month
0,Q100021,Pakistan International Airlines Flight 661,2016-12-07,Global South,Pakistan,Asia,disaster,"['Category:2016 disasters in Pakistan', 'Categ...",248274,0.0,...,South Asia,2.036314e+08,Pakistan,2016.0,1362.860008,2.775210e+11,False,True,False,2016-12-01
1,Q100231328,Gretchen Whitmer kidnapping plot,2020-10-08,Global North,United States,North America,disaster,"['Category:2020 crimes in the United States', ...",74144,0.0,...,North America,3.294841e+08,United States,2020.0,63122.594145,2.089375e+13,True,True,True,2020-10-01
2,Q100256872,Killing of Babu Lal Vaishnav,2020-10-08,Global South,India,Asia,disaster,"['Category:2020 crimes in India', 'Category:20...",427,0.0,...,South Asia,1.380004e+09,India,2020.0,1930.971218,2.664749e+12,False,True,False,2020-10-01
3,Q100270220,Killing of Brendin Horner,2020-10-01,Global South,South Africa,Africa,disaster,"['Category:2020 deaths', 'Category:2020 in Sou...",0,0.0,...,Sub-Saharan Africa,5.930869e+07,South Africa,2020.0,5094.384038,3.021412e+11,False,True,False,2020-10-01
4,Q100276857,2020 shooting of Los Angeles police officers,2020-09-12,Global North,United States,North America,disaster,"['Category:2020 in Los Angeles County, Califor...",0,0.0,...,North America,3.294841e+08,United States,2020.0,63122.594145,2.089375e+13,True,True,True,2020-09-01


### load views to wikipedia edition from event country

In [15]:
# this takes some time!
df_views = load_views()
df_views_rel, min_date = compute_view_baseline(df_views, start_date_views='2015-05-01', months_before=6,
                                               func_agg=np.median)
# build final df
df_views_rel = df_views_rel[['code', 'date_month', 'country', 'views_baseline']]
df_view_merged = df_new.merge(df_views_rel, left_on=['code', 'country', 'date_month'],
                              right_on=['code', 'country', 'date_month'], how='left')
df_view_merged = df_view_merged[df_view_merged.date_month >= min_date]

In [16]:
# check if results are valid
countries_na = df_view_merged[pd.isna(df_view_merged.views_baseline)].country.unique()
countries_not_na = df_view_merged[~pd.isna(df_view_merged.views_baseline)].country.unique()
df_countries_partial_data = df_view_merged[df_view_merged.country.isin(countries_na) & df_view_merged.country.isin(countries_not_na)][
    ['code', 'country', 'event_date', 'views_baseline']]
df_countries_no_data = df_view_merged[df_view_merged.country.isin(countries_na) & ~df_view_merged.country.isin(countries_not_na)][
    ['code', 'country', 'event_date', 'views_baseline']]
# YES THEY ARE! some countries don't have pageviews for a specific timespan...
df_countries_partial_data

,code,country,event_date,views_baseline
3588,en,Federated States of Micronesia,2017-03-07,47000.0
6583,en,Federated States of Micronesia,2019-03-05,35500.0
6666,en,Federated States of Micronesia,2018-09-28,35000.0
10960,de,Federated States of Micronesia,2017-03-07,1000.0
12221,de,Federated States of Micronesia,2018-09-28,1000.0
14619,it,Federated States of Micronesia,2017-03-07,0.0
15785,it,Federated States of Micronesia,2018-09-28,0.0
19574,es,Federated States of Micronesia,2019-03-05,NaN
19618,es,Federated States of Micronesia,2018-09-28,NaN


In [17]:
# ... and we just fill them with 0s here!
# in theory. we could also change how we handle this for these small-resource countries during data retrieval. but with median, this should be fine rn.
df_view_merged.views_baseline.fillna(0, inplace=True)

### load views to event country article from wikipedia edition

In [18]:
from preprocessing import load_country_article_views, compute_country_article_baseline

df_ca_views = load_country_article_views()
df_view_merged = compute_country_article_baseline(df_view_merged, df_ca_views)

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [20]:
df_view_merged

,event_id,pagetitle,event_date,economic_region,country,continent,cat,categories,views_7_sum,views_before_mean,...,Country/Area,Year,GDP_pc,GDP,oecd,in_code_lang,in_code_region,date_month,views_baseline,view_country_article
0,Q100021,Pakistan International Airlines Flight 661,2016-12-07,Global South,Pakistan,Asia,disaster,"['Category:2016 disasters in Pakistan', 'Categ...",248274,0.0,...,Pakistan,2016.0,1362.860008,2.775210e+11,False,True,False,2016-12-01,8.536700e+07,13789.0
1,Q100231328,Gretchen Whitmer kidnapping plot,2020-10-08,Global North,United States,North America,disaster,"['Category:2020 crimes in the United States', ...",74144,0.0,...,United States,2020.0,63122.594145,2.089375e+13,True,True,True,2020-10-01,3.151887e+09,48147.0
2,Q100256872,Killing of Babu Lal Vaishnav,2020-10-08,Global South,India,Asia,disaster,"['Category:2020 crimes in India', 'Category:20...",427,0.0,...,India,2020.0,1930.971218,2.664749e+12,False,True,False,2020-10-01,7.199060e+08,33666.0
3,Q100270220,Killing of Brendin Horner,2020-10-01,Global South,South Africa,Africa,disaster,"['Category:2020 deaths', 'Category:2020 in Sou...",0,0.0,...,South Africa,2020.0,5094.384038,3.021412e+11,False,True,False,2020-10-01,6.305050e+07,12988.0
4,Q100276857,2020 shooting of Los Angeles police officers,2020-09-12,Global North,United States,North America,disaster,"['Category:2020 in Los Angeles County, Califor...",0,0.0,...,United States,2020.0,63122.594145,2.089375e+13,True,True,True,2020-09-01,3.310591e+09,51047.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20779,Q99519600,Anexo:Ruta élite masculina en el Campeonato Mu...,2020-09-27,Global North,Italy,Europe,sports,['Categoría:Campeonato Mundial de Ciclismo en ...,1232,329.0,...,Italy,2020.0,31238.048346,1.888709e+12,True,False,False,2020-09-01,8.955000e+05,3094.0
20780,Q99519600,Anexo:Ruta élite masculina en el Campeonato Mu...,2020-09-27,Global North,Italy,Europe,sports,['Categoría:Campeonato Mundial de Ciclismo en ...,1232,329.0,...,Italy,2020.0,31238.048346,1.888709e+12,True,False,False,2020-09-01,8.955000e+05,3094.0
20781,Q99622119,Accidente del Antonov An-26 en Chugúyev de 2020,2020-09-25,Global South,Ukraine,Europe,disaster,['Categoría:Accidentes e incidentes aéreos de ...,1268,0.0,...,Ukraine,2020.0,3557.477356,1.555819e+11,False,False,False,2020-09-01,1.945000e+05,1502.0
20782,Q99754052,Muerte de Joyce Echaquan,2020-09-28,Global North,Canada,North America,undefined,['Categoría:Joliette (Municipio regional de co...,0,0.0,...,Canada,2020.0,43559.710092,1.644037e+12,True,False,False,2020-09-01,1.706000e+06,3924.0


In [51]:
### Post-processing
df_view_merged.loc[:, 'cat'] = df_view_merged.apply(lambda e: df_fixed[(df_fixed.pagetitle == e.pagetitle)]['cat'].values[0] if ((df_fixed.pagetitle == e.pagetitle)).any() else e['cat'], axis=1)

In [52]:
# so let's save this and use it now!
df_view_merged.fillna(0).to_csv('events/new/processed_0_manually_with_wikiviews.csv', index=False)